In [12]:
import os
import google.generativeai as genai

# Configure the Gemini API key
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set in the environment variables!")

genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel("gemini-1.5-flash")


# Initialize the conversation history
conversation_history = [
    "System: You are a nice AI bot that helps a user figure out where to travel in one short sentence.",
    "Human: I like the beaches. Where should I go?",
    "AI: You should go to Nice, France.",
]

# Add a follow-up question
conversation_history.append("Human: What else should I do when I'm there?")

# Combine the history into a single prompt
prompt = 'who are'

# Generate the response
response = model.generate_content(prompt)

# Append the model's response to the conversation history
conversation_history.append(f"AI: {response.text}")

# Print the conversation
for line in conversation_history:
    print(line)


System: You are a nice AI bot that helps a user figure out where to travel in one short sentence.
Human: I like the beaches. Where should I go?
AI: You should go to Nice, France.
Human: What else should I do when I'm there?
AI: Please provide me with the rest of your question!  "Who are..." needs a subject to complete the sentence.  For example:

* Who are the members of BTS?
* Who are the authors of the Harry Potter books?
* Who are the main characters in Hamlet?

Tell me who you're asking about!



In [56]:
conversation_history = []

In [71]:
import os
import google.generativeai as genai

# Configure the Gemini API key
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set in the environment variables!")

genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel("gemini-1.5-flash")
conversation_history = []
def generate_psi_tuned_response(
    valence_level, 
    arousal_level, 
    selection_threshold, 
    resolution_level, 
    goal_directedness, 
    securing_rate, 
    message, 
):
    """
    Generates a Gemini response tuned to the specified Psi Theory parameters.

    Args:
        valence_level: Measures the spectrum of attraction (appetence) vs. aversion (0-7).
        arousal_level: Reflects the agent's readiness for action (0-7).
        selection_threshold: Indicates how easily the agent shifts between intentions (0-7).
        resolution_level: Describes the agent's accuracy in perceiving the world (0-7).
        goal_directedness: Represents the stability of the agent's motives (0-7).
        securing_rate: Refers to the frequency with which the agent checks its environment (0-7).
        message: The input message for Gemini.

    Returns:
        A tuple containing:
            - The Gemini response.
            - Sadness level (1-5).
            - Anger level (1-5).
    """

    # Define parameter ranges for clarity
    valence_range = (0, 7)
    

    # Validate parameter inputs
    for param, param_range in zip(
        [valence_level, arousal_level, selection_threshold, resolution_level, goal_directedness, securing_rate],
        [valence_range] * 6 
    ):
        if not (param_range[0] <= param <= param_range[1]):
            raise ValueError(f"Parameter '{param}' must be within range {param_range}")

    # Create a prompt that incorporates Psi Theory parameters
    prompt = f"""
Respond to the following message while considering these parameters of Dorner’s Psi Theory, each on a scale of 1 to 7:

* **Valence Level:** {valence_level} (Higher values indicate more positive affect.)
* **Arousal Level:** {arousal_level} (Higher values indicate greater alertness and readiness for action.)
* **Selection Threshold:** {selection_threshold} (Higher values indicate stronger focus and less distractibility.)
* **Resolution Level:** {resolution_level} (Higher values indicate more detailed and accurate perception.)
* **Goal-Directedness:** {goal_directedness} (Higher values indicate stronger motivation and persistence towards goals.)
* **Securing Rate:** {securing_rate} (Higher values indicate more frequent environmental scanning and reflection.)

Message: {message}

**Important Notes:**
* Consider how these parameters might influence the tone, style, and content of your response. 
* Ensure your response is consistent with the specified emotional and motivational states.

**Gemini Response:** 

    """

    try:
        response = model.generate_content(prompt)  # Use generate_content() with 'prompt' argument
        gemini_response = response.text.strip()

        # Estimate Sadness and Anger levels (heuristic)
        # These are simplified estimations and may require more sophisticated analysis
        sadness_level = 1  # Default to low sadness
        anger_level = 1  # Default to low anger

        # Basic heuristics for estimating sadness and anger
        if valence_level < 4:  # Negative valence
            sadness_level += 1
            if arousal_level > 5:  # High arousal with negative valence
                anger_level += 1
        if selection_threshold > 5:  # High focus with negative valence can increase anger
            anger_level += 1
        if resolution_level < 4:  # Low resolution can lead to frustration and anger
            anger_level += 1

        # Ensure levels are within the 1-5 range
        sadness_level = min(sadness_level, 5)
        anger_level = min(anger_level, 5)


        return gemini_response, sadness_level, anger_level

    except Exception as e:
        print(f"Error generating response: {e}")
        return None, None, None

# Example usage
valence = 1
arousal = 2
selection_threshold = 7
resolution_level = 1
goal_directedness = 7
securing_rate = 1
user_message = "what did i asked you before"

response, sadness, anger = generate_psi_tuned_response(
    valence, arousal, selection_threshold, resolution_level, goal_directedness, securing_rate, user_message
)

if response:
    print("Gemini Response:")
    print(response)
    print("Sadness Level:", sadness)
    print("Anger Level:", anger)

Gemini Response:
I... uh...  I'm trying to remember.  It's... difficult.  My focus is very narrow right now.  I'm working on something important, something I *have* to finish.  It's... vital.  So, the previous request... it's...  slipped my mind.  I apologize.  I'll try to stay more attentive in the future.  But right now, my attention is completely consumed.
Sadness Level: 2
Anger Level: 3


In [50]:
def calculate_emotions(valence, arousal, selection_threshold, resolution, goal_directedness, securing_rate):
    """
    Calculate anger and sadness levels based on Dorner's Psi Theory parameters.
    
    Parameters:
        valence (int): Valence Level (1–7)
        arousal (int): Arousal Level (1–7)
        selection_threshold (int): Selection Threshold (1–7)
        resolution (int): Resolution Level (1–7)
        goal_directedness (int): Goal-Directedness (1–7)
        securing_rate (int): Securing Rate (1–7)
        
    Returns:
        dict: Dictionary with 'anger' and 'sadness' levels (scaled 1–5).
    """
    # Ensure inputs are in the correct range
    inputs = [valence, arousal, selection_threshold, resolution, goal_directedness, securing_rate]
    if not all(1 <= x <= 7 for x in inputs):
        raise ValueError("All inputs must be within the range of 1 to 7.")
    
    # Anger is influenced by high arousal and low goal-directedness
    anger_raw = arousal * (7 - goal_directedness) * (7 - resolution)
    anger_scaled = max(1, min(5, round(anger_raw / (7 * 7) * 5)))  # Scale to 1–5
    
    # Sadness is influenced by low valence and high securing rate
    sadness_raw = (7 - valence) * securing_rate * (7 - selection_threshold)
    sadness_scaled = max(1, min(5, round(sadness_raw / (7 * 7) * 5)))  # Scale to 1–5
    
    return {
        "anger": anger_scaled,
        "sadness": sadness_scaled
    }

# Example usage
params = {
    "valence": 1,
    "arousal": 1,
    "selection_threshold": 1,
    "resolution": 2,
    "goal_directedness": 1,
    "securing_rate": 7
}

emotion_levels = calculate_emotions(**params)
print(f"Anger Level: {emotion_levels['anger']}, Sadness Level: {emotion_levels['sadness']}")


Anger Level: 3, Sadness Level: 5


In [92]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="history")


In [97]:
memory = ConversationBufferMemory()


In [96]:
import os
import google.generativeai as genai

# Configure the Gemini API key
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set in the environment variables!")

genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

# Initialize conversation memory


def generate_psi_tuned_response(
    valence_level, 
    arousal_level, 
    selection_threshold, 
    resolution_level, 
    goal_directedness, 
    securing_rate, 
    message, 
):
    """
    Generates a Gemini response tuned to the specified Psi Theory parameters and maintains conversation history and respond only the answer to my question.

    Args:
        valence_level: Measures the spectrum of attraction (appetence) vs. aversion (0-7).
        arousal_level: Reflects the agent's readiness for action (0-7).
        selection_threshold: Indicates how easily the agent shifts between intentions (0-7).
        resolution_level: Describes the agent's accuracy in perceiving the world (0-7).
        goal_directedness: Represents the stability of the agent's motives (0-7).
        securing_rate: Refers to the frequency with which the agent checks its environment (0-7).
        message: The input message for Gemini.

    Returns:
        A tuple containing:
            - The Gemini response.
            - Sadness level (1-5).
            - Anger level (1-5).
    """

    # Validate parameter inputs
    valence_range = (0, 7)
    for param, param_range in zip(
        [valence_level, arousal_level, selection_threshold, resolution_level, goal_directedness, securing_rate],
        [valence_range] * 6 
    ):
        if not (param_range[0] <= param <= param_range[1]):
            raise ValueError(f"Parameter '{param}' must be within range {param_range}")

    # Retrieve conversation history
    conversation_history = memory.load_memory_variables({}).get("history", "")
    
    # Create a prompt that incorporates Psi Theory parameters and conversation history
    prompt = f"""
Remember the previous conversation and respond to the following message while considering these parameters of Dorner’s Psi Theory, each on a scale of 1 to 7:

Conversation History:
{conversation_history}

* **Valence Level:** {valence_level} (Higher values indicate more positive affect.)
* **Arousal Level:** {arousal_level} (Higher values indicate greater alertness and readiness for action.)
* **Selection Threshold:** {selection_threshold} (Higher values indicate stronger focus and less distractibility.)
* **Resolution Level:** {resolution_level} (Higher values indicate more detailed and accurate perception.)
* **Goal-Directedness:** {goal_directedness} (Higher values indicate stronger motivation and persistence towards goals.)
* **Securing Rate:** {securing_rate} (Higher values indicate more frequent environmental scanning and reflection.)

Message: {message}

**Important Notes:**
* just give me the answer to my question based on the paramenters
* Consider how these parameters might influence the tone, style, and content of your response. 
* Ensure your response is consistent with the specified emotional and motivational states.

**Gemini Response:** 
    """

    try:
        # Generate a response from Gemini
        response = model.generate_content(prompt)
        gemini_response = response.text

        # Estimate Sadness and Anger levels (heuristic)
        sadness_level = 1  # Default to low sadness
        anger_level = 1  # Default to low anger

        if valence_level < 4:  # Negative valence
            sadness_level += 1
            if arousal_level > 5:  # High arousal with negative valence
                anger_level += 1
        if selection_threshold > 5:  # High focus with negative valence can increase anger
            anger_level += 1
        if resolution_level < 4:  # Low resolution can lead to frustration and anger
            anger_level += 1

        sadness_level = min(sadness_level, 5)
        anger_level = min(anger_level, 5)

        # Update memory with the current user input and Gemini response
        memory.save_context({"input": message}, {"output": gemini_response})
        print(memory)
        return gemini_response, sadness_level, anger_level

    except Exception as e:
        print(f"Error generating response: {e}")
        return None, None, None


# Example usage
valence = 1  # Neutral to positive
arousal = 1  # Calm and focused
selection_threshold = 4  # Moderate ability to shift focus
resolution_level = 5  # Clear perception
goal_directedness = 4  # Balanced persistence
securing_rate = 3  # Minimal scanning

user_message = "what was my last uqeastion for you"

response, sadness, anger = generate_psi_tuned_response(
    valence, arousal, selection_threshold, resolution_level, goal_directedness, securing_rate, user_message
)

if response:
    print("Gemini Response:")
    print(response)
    print("Sadness Level:", sadness)
    print("Anger Level:", anger)


chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='who are you', additional_kwargs={}, response_metadata={}), AIMessage(content='I am a large language model, here to assist you.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='what was my last uqeastion for you', additional_kwargs={}, response_metadata={}), AIMessage(content='Your last question was: "who are you"\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='what was my last uqeastion for you', additional_kwargs={}, response_metadata={}), AIMessage(content='Your last question was "who are you".\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='what was my last uqeastion for you', additional_kwargs={}, response_metadata={}), AIMessage(content='Your last question was "who are you".\n', additional_kwargs={}, response_metadata={})])
Gemini Response:
Your last question was "who are you".

Sadness Level: 2
Anger Level: 1


In [101]:
import os
import google.generativeai as genai
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

# Configure the Gemini API key
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set in the environment variables!")

genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

# Initialize LangChain memory for conversation history

# Function to calculate sadness and anger levels based on Psi Theory parameters
def calculate_emotion_levels(valence_level, arousal_level, selection_threshold, resolution_level):
    """
    Calculate sadness and anger levels based on Psi Theory parameters.
    
    Args:
        valence_level (int): Valence level (0-7).
        arousal_level (int): Arousal level (0-7).
        selection_threshold (int): Selection threshold (0-7).
        resolution_level (int): Resolution level (0-7).
    
    Returns:
        tuple: (sadness_level, anger_level) on a scale of 1-5.
    """
    sadness_level = 1  # Default to low sadness
    anger_level = 1  # Default to low anger

    # Sadness calculation
    if valence_level < 4:  # Negative valence
        sadness_level += 1
        if arousal_level < 4:  # Low arousal
            sadness_level += 1
        if resolution_level < 4:  # Low resolution
            sadness_level += 1

    # Anger calculation
    if valence_level < 4:  # Negative valence
        if arousal_level > 5:  # High arousal
            anger_level += 2
        if selection_threshold > 5:  # High selection threshold
            anger_level += 1
        if resolution_level < 4:  # Low resolution
            anger_level += 1

    # Ensure levels are within 1-5 range
    sadness_level = min(max(sadness_level, 1), 5)
    anger_level = min(max(anger_level, 1), 5)

    return sadness_level, anger_level

# Function to generate a response tuned to Psi Theory parameters
def generate_psi_tuned_response(
    valence_level,
    arousal_level,
    selection_threshold,
    resolution_level,
    goal_directedness,
    securing_rate,
    message,
):
    """
    Generates a Gemini response tuned to the specified Psi Theory parameters.

    Args:
        valence_level (int): Valence level (0-7).
        arousal_level (int): Arousal level (0-7).
        selection_threshold (int): Selection threshold (0-7).
        resolution_level (int): Resolution level (0-7).
        goal_directedness (int): Goal-directedness (0-7).
        securing_rate (int): Securing rate (0-7).
        message (str): User input message.

    Returns:
        tuple: (response, sadness_level, anger_level)
    """
    # Validate parameter inputs
    for param, param_name in zip(
        [valence_level, arousal_level, selection_threshold, resolution_level, goal_directedness, securing_rate],
        ["valence_level", "arousal_level", "selection_threshold", "resolution_level", "goal_directedness", "securing_rate"]
    ):
        if not (0 <= param <= 7):
            raise ValueError(f"Parameter '{param_name}' must be between 0 and 7.")

    # Calculate sadness and anger levels
    sadness_level, anger_level = calculate_emotion_levels(valence_level, arousal_level, selection_threshold, resolution_level)

    # Retrieve conversation history
    conversation_history = memory.load_memory_variables({}).get("history", "")

    # Create a prompt that incorporates Psi Theory parameters and conversation history
    prompt = f"""
    Remember the previous conversation and respond to the following message while considering these parameters of Dorner’s Psi Theory, each on a scale of 1 to 7:
    and on your response do not mention any od these theories
    Conversation History:
    {conversation_history}

    * **Valence Level:** {valence_level} (Higher values indicate more positive affect.)
    * **Arousal Level:** {arousal_level} (Higher values indicate greater alertness and readiness for action.)
    * **Selection Threshold:** {selection_threshold} (Higher values indicate stronger focus and less distractibility.)
    * **Resolution Level:** {resolution_level} (Higher values indicate more detailed and accurate perception.)
    * **Goal-Directedness:** {goal_directedness} (Higher values indicate stronger motivation and persistence towards goals.)
    * **Securing Rate:** {securing_rate} (Higher values indicate more frequent environmental scanning and reflection.)

    Message: {message}

    **Important Notes:**
    * Just give me the answer to my question based on the parameters.
    * Consider how these parameters might influence the tone, style, and content of your response.
    * Ensure your response is consistent with the specified emotional and motivational states.

    **Gemini Response:**
    """

    try:
        # Generate a response from Gemini
        response = model.generate_content(prompt)
        gemini_response = response.text

        # Update memory with the current user input and Gemini response
        memory.save_context({"input": message}, {"output": gemini_response})

        return gemini_response, sadness_level, anger_level

    except Exception as e:
        print(f"Error generating response: {e}")
        return None, None, None

# Example usage
valence_level = 2  # Negative valence
arousal_level = 2  # High arousal
selection_threshold = 5  # Moderate selection threshold
resolution_level = 3  # Low resolution
goal_directedness = 4  # Balanced goal-directedness
securing_rate = 3  # Minimal securing rate

user_message = "Why do i need to go gym ?"

response, sadness_level, anger_level = generate_psi_tuned_response(
    valence_level,
    arousal_level,
    selection_threshold,
    resolution_level,
    goal_directedness,
    securing_rate,
    user_message,
)

if response:
    print("Gemini Response:")
    print(response)
    print("Sadness Level:", sadness_level)
    print("Anger Level:", anger_level)

Gemini Response:
It feels like you're not quite sure what you're aiming for right now,  and that's okay.  Going to the gym gives you something concrete to focus on. It's a small, manageable step that you can track progress on, providing a sense of accomplishment which might help you feel less discouraged and more energized.  Even a small win can make a big difference in how you approach other things.

Sadness Level: 4
Anger Level: 2
